# 1 Introduction



### 1.0.1 Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

### 1.0.2 Data

In [6]:
players = pd.read_csv('data/processed/players_final.csv')
teams = pd.read_csv('data/processed/teams_final.csv')
teams.drop('Unnamed: 0', axis = 1, inplace = True)

## 1.1 [Webscraping](https://github.com/gcox32/NBA_Player_Value/tree/master/webscrape)

### 1.1.1 [pergame Webscrape](https://github.com/gcox32/NBA_Player_Value/blob/master/webscrape/1.1-per_game%20Webscrape.ipynb)

### 1.1.2 [per100 Webscrape](https://github.com/gcox32/NBA_Player_Value/blob/master/webscrape/1.2-per_100%20Webscrape.ipynb)

### 1.1.3 [shooting Webscrape](https://github.com/gcox32/NBA_Player_Value/blob/master/webscrape/1.3-shooting%20Webscrape.ipynb)

### 1.1.4 [salary data Webscrape](https://github.com/gcox32/NBA_Player_Value/blob/master/webscrape/1.4-salary%20Webscrape.ipynb)

### 1.1.5 [team data Webscrape](https://github.com/gcox32/NBA_Player_Value/blob/master/webscrape/1.5-team%20Webscrape.ipynb)

# 2 Data Wrangling and LDA

## 2.1 [Data Wrangling](https://github.com/gcox32/NBA_Player_Value/blob/master/2.0-Player-level%20Data%20Wrangling.ipynb)

## 2.2 [Player-level LDA](https://github.com/gcox32/NBA_Player_Value/blob/master/2.1-Player-level%20LDA.ipynb)

# 3 Exploratory Data Analysis and Inferential Statistics

### 3.0.1 [lib\](https://github.com/gcox32/NBA_Player_Value/tree/master/lib)

## 3.1 [Team-level EDA](https://github.com/gcox32/NBA_Player_Value/blob/master/3.1-Team-level%20EDA.ipynb)

## 3.2 [Player Salary EDA](https://github.com/gcox32/NBA_Player_Value/blob/master/3.2-Salary%20EDA.ipynb)

## 3.3 [Inferential Statistics](https://github.com/gcox32/NBA_Player_Value/blob/master/3.3-Inferential%20Statistics.ipynb)

# 4 Conclusions and Next Steps